In [20]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

from sklearn.model_selection import train_test_split 

In [21]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'BAQ'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=20)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 1

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [22]:
clear_buyers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",136016,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [23]:
clear_buyers["last_order_date"] = pd.to_datetime(clear_buyers["last_order_date"]) 
clear_buyers['validate'] = clear_buyers.last_order_date >= customer_date_desde
clear_buyers.head()

,customer_id,last_order_date,recency_days,registered_city,validate
0,286142696,2023-01-11,1,BOG,True
1,286109806,2023-01-11,1,BOG,True
2,285899630,2023-01-11,1,BOG,True
3,285928111,2023-01-11,1,BHZ,True
4,285883881,2023-01-11,1,VCP,True


In [24]:
query = """
WITH
info AS (
SELECT DISTINCT
    s.site_identifier_value as region_code,
    --ffg.close_date,
    bo.customer_id,
    --bs.addl_product_id as padre_sku_id,
    --bs2.name AS card_name,
    --bcat2.name AS category,
    bcat.name AS subcat,
    COUNT(DISTINCT EXTRACT(WEEK FROM ffg.close_date)) AS count_weeks,
    --baid.adjustment_reason,
    SUM ( (boi.quantity * foi.step_unit * boi.sale_price)/{tipo_cambio}) AS gmv_usd
    --SUM ( COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit)/{tipo_cambio}, 0) ) AS discount_applied,
    --SUM ( boi.quantity* foi.step_unit ) AS cant

FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs2        ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"              bcat2      ON bcx.category_id = bcat2.category_id
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

WHERE 
   DATE(ffg.close_date) >= current_date - interval '10 week' 
   AND DATE(ffg.close_date) <=  current_date
   AND bo.order_status='SUBMITTED'
   AND fo.fb_order_status_id IN (1,6,7,8) 
   AND s.site_identifier_value = '{ciudad}'
   AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
   AND bop.archived = 'N'
   AND (fot.name IS NULL OR fot.name <> 'REFUND')
   -- FIX SUPER DESCUENTOS
   AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
   AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
GROUP BY 1,2,3--,5,6,7
)

SELECT 
*
FROM info
WHERE count_weeks > 5
""".format(tipo_cambio=t_cambio, ciudad=city_code)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd"]] = data_ventas_aux[["gmv_usd"]].astype(float)
#data_ventas["submit_date"] = pd.to_datetime(data_ventas["submit_date"])

In [25]:
data_ventas_aux.head()

,region_code,customer_id,subcat,count_weeks,gmv_usd
0,BAQ,207464985,Harinas & Mezclas,10,2150.188293
1,BAQ,188016019,Harinas & Mezclas,10,9519.382414
2,BAQ,15393078,Tubérculos,10,349.536015
3,BAQ,157495480,Azúcar & Endulzantes,9,45.544491
4,BAQ,59929366,Aceites & Grasas,9,443.523569


In [26]:
data_ventas_aux.subcat.unique()

array(['Harinas & Mezclas', 'Tubérculos', 'Azúcar & Endulzantes',
       'Aceites & Grasas', 'Pollo Congelado',
       'Detergente, Jabón & Lavalozas', 'Verduras', 'Vasos', 'Frutas',
       'Arroz', 'Papel & Toallas', 'Gaseosas', 'Bolsas', 'Granos',
       'Cerdo Congelado', 'Res Congelado', 'Contenedores', 'Huevos',
       'Cremas', 'Café, Chocolate e Infusiones', 'Alimentos Congelados',
       'Leches', 'Salsas', 'Sal & Sazonadores',
       'Pescados & Mariscos Congelado', 'Empaque & Envoltura',
       'Res Fresco', 'Platos & Bandejas', 'Embutidos',
       'Cubiertos & Pitillos', 'Desinfectantes & Sanitizantes', 'Pasta',
       'Derivados Lácteos', 'Pulpas de Fruta', 'Jugos',
       'Implementos de Aseo', 'Panadería & Repostería', 'Quesos',
       'Cerdo Fresco', 'Aguas', 'Tés', 'Enlatados & Conservas'],
      dtype=object)

In [27]:
subcats = ['Harinas & Mezclas','Azúcar & Endulzantes','Pollo Congelado','Tubérculos',
           'Arroz','Aceites & Grasas','Alimentos Congelados','Gaseosas','Huevos','Contenedores']

In [28]:
data_ventas = data_ventas_aux[data_ventas_aux.subcat.isin(subcats)].copy()

In [29]:
f = data_ventas.groupby("subcat", as_index=False).agg({'customer_id':[lambda x: x.nunique(),lambda x: list(x.unique())]}).droplevel(0, axis="columns")
f.columns = ['subcat', 'customers', 'list_customers']
f

,subcat,customers,list_customers
0,Aceites & Grasas,938,"[59929366, 94434353, 240185617, -213111, 98665..."
1,Alimentos Congelados,101,"[128425420, 57888163, -213672, 233763147, 8760..."
2,Arroz,562,"[180889984, 257208695, 53014568, 180890655, 61..."
3,Azúcar & Endulzantes,204,"[157495480, -220531, 130031964, 216522233, 248..."
4,Contenedores,262,"[215489158, 252511547, 234171607, 209398547, -..."
5,Gaseosas,75,"[177827626, 118084620, 168393274, 201609316, 4..."
6,Harinas & Mezclas,305,"[207464985, 188016019, 58612251, 187091618, 16..."
7,Huevos,98,"[259034902, 174691713, 108367168, 236842161, 1..."
8,Pollo Congelado,474,"[115370919, 259047662, 245602783, 129172944, 1..."
9,Tubérculos,464,"[15393078, 42147926, 74083130, 17417844, 11889..."


In [30]:
def create_groups(subcat):
    s = f[f.subcat == subcat].list_customers.to_numpy()[0]
    
    d = data_ventas[(data_ventas.customer_id.isin(s)) & (data_ventas.subcat == subcat)].groupby("customer_id").gmv_usd.mean().reset_index()
    
    customer_control, customer_test = train_test_split(s, test_size = 0.50)
    
    while abs(1-(d[d.customer_id.isin(customer_control)]['gmv_usd'].mean()/d[d.customer_id.isin(customer_test)]['gmv_usd'].mean())) > 0.01:
        customer_control, customer_test = train_test_split(s, test_size = 0.50)
        
    all_users = clear_buyers[(~clear_buyers.customer_id.isin(customer_test)) & (clear_buyers.registered_city == city_code)].customer_id.unique().tolist()
    
    segt = {'name':"SH_Proyecto_Elasticidad"+todays_date+"_"+city_code+"_"+subcat, 'customersIds': list(all_users)}
    
    print(subcat, len(all_users), len(customer_test))
    return customer_control, customer_test, segt

In [31]:
global_segment_list = []
r= {}

for subcat in f.subcat:
    control, test, segment = create_groups(subcat)
    
    r['CONTROL_'+todays_date+"_"+city_code+"_"+subcat] = control
    r['TEST_'+todays_date+"_"+city_code+"_"+subcat] = test
    
    global_segment_list.append(segment) 

df_test_control = pd.DataFrame.from_dict(r, orient='index').T

Aceites & Grasas 3357 469
Alimentos Congelados 3721 51
Arroz 3582 281
Azúcar & Endulzantes 3680 102
Contenedores 3641 131
Gaseosas 3731 38
Harinas & Mezclas 3626 153
Huevos 3728 49
Pollo Congelado 3587 237
Tubérculos 3610 232


In [32]:
len(clear_buyers[clear_buyers.registered_city == 'BAQ'])

3765

In [60]:
import requests
import json

url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [61]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]
sgt

[(114654, 'SH_Proyecto_Elasticidad2023-01-11_BAQ_Aceites & Grasas'),
 (114658, 'SH_Proyecto_Elasticidad2023-01-11_BAQ_Alimentos Congelados'),
 (114659, 'SH_Proyecto_Elasticidad2023-01-11_BAQ_Arroz'),
 (114656, 'SH_Proyecto_Elasticidad2023-01-11_BAQ_Azúcar & Endulzantes'),
 (114657, 'SH_Proyecto_Elasticidad2023-01-11_BAQ_Contenedores'),
 (114655, 'SH_Proyecto_Elasticidad2023-01-11_BAQ_Gaseosas'),
 (114662, 'SH_Proyecto_Elasticidad2023-01-11_BAQ_Harinas & Mezclas'),
 (114660, 'SH_Proyecto_Elasticidad2023-01-11_BAQ_Huevos'),
 (114663, 'SH_Proyecto_Elasticidad2023-01-11_BAQ_Pollo Congelado'),
 (114661, 'SH_Proyecto_Elasticidad2023-01-11_BAQ_Tubérculos')]